In [1]:
# coding=utf-8
import codecs
import sys
import re
import pandas as pd
import numpy as np
from collections import deque  
relation2id = {
"Other": 0,
"Cause-Effect": 1,
"Instrument-Agency":2,
"Product-Producer":3,
"Content-Container":4,
"Entity-Origin":5,
"Entity-Destination":6,
"Component-Whole":7,
"Member-Collection":8,
"Message-Topic":9
}
datas = []
labels = []
entity1 = []
entity2 = []
with codecs.open('/Users/denhiroshi/Downloads/ChineseNRE-master/data/SemEval2010_task8_all_data/TRAIN_FILE.TXT','r','utf-8') as tra:
    linenum = 0
    for line in tra:
        linenum+=1
        if linenum%4==1:
            line = line.split('\t')[1]
            word_arr = line[1:-4].split()
            for index in range(len(word_arr)):
                if "<e1>" in word_arr[index]:
                    entity1.append(index)
                elif "<e2>" in word_arr[index]:
                    entity2.append(index)

            line = line.replace("<e1>","")
            line = line.replace("</e1>","")
            line = line.replace("<e2>","")
            line = line.replace("</e2>","")
            line = re.sub(r'[^\w\s]','',line)
            datas.append(line[0:-2])
        elif linenum%4==2:
            if line=="Other\r\n":
                labels.append(0)
            else:
                line = line.split('(')
                labels.append(relation2id[line[0]])
            
        else:
            continue
print(len(datas),len(labels),len(entity1),len(entity2))
print(datas[0],labels[0],entity1[0],entity2[0])

8000 8000 8000 8000
The system as described above has its greatest application in an arrayed configuration of antenna elements 7 12 15


In [13]:
from torchtext import data
from torchtext.vocab import Vectors
from torch.nn import init
from tqdm import tqdm
import numpy as np
import os
from random import shuffle
import pandas as pd
import torch
import time
import random

In [14]:
write_csv = []
for i in range(len(datas)):
    write_csv.append((str(i), datas[i], labels[i]))

In [15]:
import csv
headers = ['id', 'Text', 'label']
with open('glove_WV_2010_task8.csv','w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(write_csv)

In [16]:
def get_dataset(csv_data, text_field, label_field, test=False):

    fields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("Text", text_field), ("label", label_field)]       
    examples = []

    if test:
        # 如果为测试集，则不加载label
        for text in tqdm(csv_data['Text']):
            examples.append(data.Example.fromlist([None, text, None], fields))
    else:
        for text, label in tqdm(zip(csv_data['Text'], csv_data['label'])):
            examples.append(data.Example.fromlist([None, text, label], fields))
    return examples, fields
tokenize = lambda x: x.split()
train_data = pd.read_csv('glove_WV_2010_task8.csv')

TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True)
LABEL = data.Field(sequential=False, use_vocab=False)

# 得到构建Dataset所需的examples和fields
train_examples, train_fields = get_dataset(train_data, TEXT, LABEL)
# 构建Dataset数据集
train = data.Dataset(train_examples, train_fields)
train

8000it [00:00, 49304.44it/s]


In [17]:
train.examples[0].Text

['the',
 'system',
 'as',
 'described',
 'above',
 'has',
 'its',
 'greatest',
 'application',
 'in',
 'an',
 'arrayed',
 'configuration',
 'of',
 'antenna',
 'elements']

In [18]:
TEXT.build_vocab(train)
vocab = TEXT.vocab
vocab.freqs.most_common(10)

[('the', 13014),
 ('of', 5488),
 ('a', 5376),
 ('and', 3372),
 ('in', 3071),
 ('to', 2590),
 ('is', 1645),
 ('from', 1446),
 ('was', 1439),
 ('by', 1267)]

In [19]:
cache = '.vector_cache'
if not os.path.exists(cache):
    os.mkdir(cache)
vectors = Vectors(name='/Users/denhiroshi/Downloads/glove/glove.6B.200d.txt', cache=cache)
TEXT.build_vocab(train, vectors=vectors)
torch.sum(TEXT.vocab.vectors != 0)/200,TEXT.vocab.vectors.shape
from torchtext.data import Iterator, BucketIterator
train_iter = Iterator(dataset = train, batch_size=64, device=-1, shuffle=False, sort_within_batch=False, repeat=False)
weight_matrix = TEXT.vocab.vectors

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [20]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
weight_matrix = TEXT.vocab.vectors

In [21]:
class WV(nn.Module):

    def __init__(self):
        super(WV, self).__init__()
        self.word_embeddings = nn.Embedding(len(TEXT.vocab), 200)  # embedding之后的shape: torch.Size([200, 8, 300])
        self.word_embeddings.weight.data.copy_(weight_matrix)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        return embeds

In [23]:
model = WV()
emm = []
model.train()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
loss_funtion = F.cross_entropy
for step, batch_train in enumerate(train_iter):
    predicted = model(batch_train.Text)
    emm.append(predicted)
len(emm)

125

In [24]:
MAX_LENGTH = 0
for i in range(len(emm)):
    MAX_LENGTH = max(MAX_LENGTH, emm[i].shape[0])
MAX_LENGTH

85

In [25]:
save = []
for temp in emm:
    for i in range(temp.shape[1]):
        now = temp[:,i,:]
        now = np.array(now.tolist())
        if now.shape[0] < MAX_LENGTH:
            add = MAX_LENGTH - now.shape[0]
            adds = np.zeros((add,200))
            now = np.concatenate((now, adds))
        save.append(now)

In [26]:
save = np.stack(save)
save.shape

(8000, 85, 200)

In [27]:
save.tofile('tensors_2010'+'.dat',format='%s')   

In [28]:
label = np.stack(list(labels))
entity1 = np.stack(list(entity1))
entity2 = np.stack(list(entity2))

In [29]:
label,entity1,entity2

(array([7, 0, 2, ..., 6, 0, 3]),
 array([12,  1,  1, ...,  7, 17,  1]),
 array([15,  9,  7, ..., 10, 18,  5]))

In [30]:
label.tofile('tensors_2010_labels'+'.dat',format='%s')
entity1.tofile('tensors_2010_entity1'+'.dat',format='%s')
entity2.tofile('tensors_2010_entity2'+'.dat',format='%s')